In [1]:
from appgeopy import *
from my_packages import *

In [2]:
script_folder = os.getcwd()

In [3]:
gwl_hdf5_file = r"D:\1000_SCRIPTS\002_PostQE_Scripts\20240819_Save_GWL_HDF5\20240826_GWL_MonitoringData_Choushuixi.h5"

with h5py.File(gwl_hdf5_file, "r") as hdf5_file:
    available_datasets = gwatertools.h5pytools.list_datasets(hdf5_file)

hdf5_file.close()

In [4]:
new_data_mainfolder = r"D:\VINHTRUONG\004_MODELING\001_STUDY_AREA\GroundwaterObservation\@DOWNLOAD_WRA_GWOB_YEARBOOK_PROJECT\@groundwater_level_PDF\GW_DATA_gweb.wra.gov.tw\COMBINE\_2020__2023"

os.chdir(new_data_mainfolder)
# - - - - -
stations_fld = [f for f in os.listdir() if os.path.isdir(f)]
# - - - - -
new_data = {}
error_log = {}
# - - - - -
# select_station = stations_fld[0]
for select_station in tqdm(stations_fld):
    try:
        ename, cname, abbrev = select_station.upper().split("_")

        temp = {ename: {}}
        # - - - - -
        files_byStation = glob(select_station + "\\*.xlsx")
        # - - - - -
        # select_file = files_byStation[0]
        for select_file in files_byStation:
            wellcode = os.path.basename(select_file).split(".")[0]
            # - - - - -
            # extract the monitoring data from HDF5 file
            # based on the station and well code
            df_fromHDF5 = gwatertools.h5pytools.export_data_to_dataframe(
                file_name=gwl_hdf5_file,
                location_name=ename,
                sensor_name=wellcode,
            )
            df_fromHDF5 = df_fromHDF5.set_index("datetime")
            # - - - - -
            # get the monitoring data from excel file
            df_NewData = pd.read_excel(
                select_file, parse_dates=[0], index_col=[0]
            )
            # - - - - -
            # fill the missing data from
            df_fromHDF5_filled = df_fromHDF5.fillna(df_NewData)

            temp[ename].update({wellcode: df_fromHDF5_filled["value"]})

        new_data.update(temp)
    except Exception as e:
        error_log.update({select_station: e})
        pass

 13%|██████████▍                                                                       | 11/86 [00:01<00:06, 10.76it/s]

Dataset 'DAPI/09080311' not found. Available datasets: ['ANHE/10070111', 'ANHE/10070121', 'ANHE/10070131', 'ANHE/10070141', 'ANHE/date', 'ANNAN/09140111', 'ANNAN/09140112', 'ANNAN/09140121', 'ANNAN/09140122', 'ANNAN/date', 'BEIGANG/09060111', 'BEIGANG/09060112', 'BEIGANG/09060121', 'BEIGANG/09060122', 'BEIGANG/date', 'BOZI/09180111', 'BOZI/09180121', 'BOZI/09180131', 'BOZI/date', 'CAICUO/09180211', 'CAICUO/09180221', 'CAICUO/date', 'CHENGUANG/09060211', 'CHENGUANG/09060212', 'CHENGUANG/date', 'CHUKOU/09100211', 'CHUKOU/09100221', 'CHUKOU/date', 'CITONG/09090211', 'CITONG/09090212', 'CITONG/09090221', 'CITONG/09090222', 'CITONG/date', 'DAGOU/09200211', 'DAGOU/09200221', 'DAGOU/date', 'DAQI/10050211', 'DAQI/10050221', 'DAQI/date', 'DINGFAN/070213M3', 'DINGFAN/date', 'DONGFANG/07010211', 'DONGFANG/07010221', 'DONGFANG/date', 'DONGGUANG/09180411', 'DONGGUANG/09180421', 'DONGGUANG/09180431', 'DONGGUANG/09180441', 'DONGGUANG/09180451', 'DONGGUANG/date', 'DONGHE/09070111', 'DONGHE/09070112', 

100%|██████████████████████████████████████████████████████████████████████████████████| 86/86 [00:11<00:00,  7.49it/s]


In [5]:
# Prepare updates_dict by transforming new data into the expected format
updates_dict = {}

for station, wellcode in new_data.items():
    well_metadata = {
        well: {
            "START_OBS": series.first_valid_index().strftime("%Y%m%d"),
            "LAST_OBS": series.last_valid_index().strftime("%Y%m%d"),
        }
        for well, series in wellcode.items()
    }
    well_data = {well:series.values for well, series in wellcode.items()}
    dates = list(wellcode.values())[0].index.strftime('%Y%m%d').tolist()

    # Prepare data for each location
    updates_dict[station] = {
        'sensor_data': well_data,
        'metadata': {},  # Metadata is left empty to avoid overwriting
        'sensor_metadata': well_metadata  # Sensor-specific metadata is also left empty
    }
    # Include the date information
    updates_dict[station]['sensor_data']['date'] = np.array(dates, dtype='S10')

In [6]:
# Update the HDF5 file using the existing function
gwatertools.h5pytools.update_hdf5_from_dict(gwl_hdf5_file, updates_dict)

Successfully updated the file 'D:\1000_SCRIPTS\002_PostQE_Scripts\20240819_Save_GWL_HDF5\20240826_GWL_MonitoringData_Choushuixi.h5'.
